## Continual Learning on the Alzheimer's Dataset

A basic CL based approach to classifying levels of Alzheimer's based on MRI scan images

Dataset Link: https://www.kaggle.com/tourist55/alzheimers-dataset-4-class-of-images

In [1]:
from avalanche.benchmarks.utils import AvalancheDataset
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import accuracy_metrics, loss_metrics
import os
import torch
from torch import nn
from torchvision import transforms
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset
from avalanche.benchmarks.generators import dataset_benchmark
from avalanche.training.strategies import EWC, LwF
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
transform=transforms.Compose([
    transforms.RandomResizedCrop(180),
    transforms.CenterCrop(180),
    transforms.Resize(255),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

In [4]:
DATA_DIR='E:/Datasets/Alzheimer_s Dataset/'

In [5]:
train_folder=ImageFolder(os.path.join(DATA_DIR,'train'),transform=transform)

In [6]:
test_folder=ImageFolder(os.path.join(DATA_DIR,'test'),transform=transform)

In [7]:
type(test_folder)

torchvision.datasets.folder.ImageFolder

In [8]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(3,6,5)
        self.pool=nn.MaxPool2d(2,2)
        self.conv2=nn.Conv2d(6,16,5)
        self.fc1=nn.Linear(16*60*60,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,4)
        self.softmax=nn.Softmax(dim=1)
        
    def forward(self,x):
        x=self.conv1(x)
        x=F.relu(x)
        x=self.pool(x)
        x=self.pool(F.relu(self.conv2(x)))
        x=torch.flatten(x,1)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.softmax(self.fc3(x))
        return x
cnn=CNN() 

In [9]:
len(test_folder)

1279

In [10]:
train_folder[0]

(tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]]),
 0)

In [11]:
class AlzheimersDataset(Dataset):
    def __init__(self, folder):
        self.folder=folder
        self.targets=folder.targets
    
    def __len__(self):
        return len(self.folder)
    
    def __getitem__(self,idx):
        return self.folder[idx]
        
        

In [12]:
train_data=AlzheimersDataset(train_folder)

In [13]:
test_data=AlzheimersDataset(test_folder)

In [14]:
AlzheimersBM=dataset_benchmark(train_datasets=[train_data],test_datasets=[test_data])

In [15]:
train_stream=AlzheimersBM.train_stream
test_stream=AlzheimersBM.test_stream

In [16]:
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger])

c:\users\ashwin\miniconda3\lib\site-packages\avalanche\training\plugins\evaluation.py:75: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  warnings.warn(


In [17]:
strategy=EWC(cnn,torch.optim.Adam(cnn.parameters()),nn.CrossEntropyLoss(),ewc_lambda=0.001,train_mb_size=32,train_epochs=10,eval_mb_size=13, evaluator=eval_plugin, device=device)

In [18]:
results=[]
for train_task in train_stream:
    print(train_task.current_experience)
    strategy.train(train_task)
    results.append(strategy.eval(test_stream))


0
-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --
0it [00:00, ?it/s]

c:\users\ashwin\miniconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


100%|████████████████████████████████████████████████████████████████████████████████| 161/161 [00:32<00:00,  4.93it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.2136
	Loss_MB/train_phase/train_stream/Task000 = 0.7451
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5071
	Top1_Acc_MB/train_phase/train_stream/Task000 = 1.0000
100%|████████████████████████████████████████████████████████████████████████████████| 161/161 [00:30<00:00,  5.26it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.2239
	Loss_MB/train_phase/train_stream/Task000 = 1.2022
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5063
	Top1_Acc_MB/train_phase/train_stream/Task000 = 1.0000
100%|████████████████████████████████████████████████████████████████████████████████| 161/161 [00:24<00:00,  6.64it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.2124
	Loss_MB/train_phase/train_stream/Task000 = 0.7550
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5

In [19]:
results

[{'Top1_Acc_MB/train_phase/train_stream/Task000': 0.0,
  'Loss_MB/train_phase/train_stream/Task000': 1.6625163555145264,
  'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.5344659246240968,
  'Loss_Epoch/train_phase/train_stream/Task000': 1.1838139302794344,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.527756059421423,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 1.1936826997198473,
  'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.527756059421423,
  'Loss_Stream/eval_phase/test_stream/Task000': 1.1936826997198473}]

In [21]:
i,l=test_data[100]

In [24]:
o=cnn(torch.unsqueeze(i,0).cuda())

In [32]:
o[0][3]

tensor(0.5818, device='cuda:0', grad_fn=<SelectBackward>)